In [46]:
from statistics import median

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor #model training
from sklearn.metrics import mean_absolute_error #model evaluation
from sklearn.impute import  SimpleImputer

In [47]:
X_full = pd.read_csv('train.csv', index_col='Id')

In [48]:
X_test_full = pd.read_csv('test.csv', index_col='Id')

In [49]:
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True) #drop missing values in the target dataframe
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

In [50]:
X = X_full.select_dtypes(exclude=['object'])
X_test= X_test_full.select_dtypes(exclude=['object'])

X_train,X_valid,y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


In [51]:
print(X_train.shape )#shape of teh feature training set
print(f"num of rows:{X_train.shape[0]}")
print(f"num of columns:{X_train.shape[1]}")

(1168, 36)
num of rows:1168
num of columns:36


In [52]:
missing_val_count_by_column = (X_train.isnull().sum())
# print(missing_val_count_by_column[missing_val_count_by_column > 0].sum())
missign_col = missing_val_count_by_column[missing_val_count_by_column > 0]
print()

In [53]:
def score_dataset(X_train, y_train, X_valid, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [54]:
#get name of column with missing values
# Get columns with missing values
cols_with_missing = []
for col in X_train.columns:
    if X_train[col].isnull().any():
        cols_with_missing.append(col)
print("Columns with missing values:", cols_with_missing)

# To also see the number of missing values in each column
missing_values = X_train.isnull().sum()
missing_values = missing_values[missing_values > 0]
print("\nNumber of missing values in each column:")
print(missing_values)

Columns with missing values: ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

Number of missing values in each column:
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


In [55]:
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)
print(score_dataset(reduced_X_train, y_train, reduced_X_valid, y_valid))

17837.82570776256


IMPUTATION OF MISSING VALUES


In [56]:
imputer = SimpleImputer() #instace of the imputer
imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(imputer.transform(X_valid))

In [57]:
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns


print(score_dataset(imputed_X_train, y_train, imputed_X_valid, y_valid))

18062.894611872147


In [58]:
final_imputer = SimpleImputer(strategy='median')
final_X_train = pd.DataFrame(final_imputer.fit_transform(X_train))
final_X_valid = pd.DataFrame(final_imputer.transform(X_valid))

In [59]:
final_X_train.columns = X_train.columns
final_X_valid.columns = X_valid.columns

In [60]:
model = RandomForestRegressor(n_estimators=100, random_state=0 )
model.fit(final_X_train, y_train)

RandomForestRegressor(n_estimators=180, random_state=0)

In [61]:
pred_valid = model.predict(final_X_valid)
print("MAE (Your approach):")
print(mean_absolute_error(y_valid, pred_valid))

MAE (Your approach):
17507.830206747843


In [ ]:
final_X_test = pd.DataFrame(final_imputer.transform(X_test))
preds_test = model.predict(final_X_test)